# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [347]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [348]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Victor Cunha\Desktop\Atividades\C-Dados\CDADOS - DP\projeto1-cdados


Carregando a base de dados com os tweets classificados manualmente:

In [349]:
train = pd.read_excel('dados_treino_CLASSIFICADO.xlsx')
train.head(5)

,Mensagem,Target
0,"Não cheguei a completar a leitura do livro, te...",2
1,"Péssimo. Qualidade horrível do papel, tamanho ...",2
2,Fiz a compra do produto e chegou tudo certo co...,0
3,"Vetusto, tanto quanto o próprio autor, comprar...",2
4,Se soubesse não teria comprado. É bom pra apre...,2


In [350]:
test = pd.read_excel('dados_teste_CLASSIFICADO.xlsx')
test.head(5)

,Mensagem,Target
0,"Livro ruim, comparado com seu antecessor! A au...",2
1,O Livro é um resumo do canal Me Poupe! O Conte...,2
2,Desnecessário comentar. Não percam tempo. Níve...,2
3,Essa encomenda não veio pelos correios e chego...,0
4,"Assim como todos os livro da Série, não curti....",2


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [351]:
import re

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [352]:
train_pos = train.loc[train['Target'] == 1, ['Mensagem']]
train_neg = train.loc[train['Target'] == 2, ['Mensagem']]
train_irr = train.loc[train['Target'] == 0, ['Mensagem']]
train_all = train.loc[:,['Mensagem']]

In [353]:
lista_train_pos = train_pos['Mensagem'].values.tolist()
lista_train_neg = train_neg['Mensagem'].values.tolist()
lista_train_irr = train_irr['Mensagem'].values.tolist()
lista_train_all = train_all['Mensagem'].values.tolist()

pos_clean = []
neg_clean = []
irr_clean = []
train_clean = []

for phrase in lista_train_pos:
    phrase_clean = cleanup(phrase)
    phrase_clean = phrase_clean.split()
    for word in phrase_clean:
        pos_clean.append(word)

for phrase in lista_train_neg:
    phrase_clean = cleanup(phrase)
    phrase_clean = phrase_clean.split()
    for word in phrase_clean:
        neg_clean.append(word)

for phrase in lista_train_irr:
    phrase_clean = cleanup(phrase)
    phrase_clean = phrase_clean.split()
    for word in phrase_clean:
        irr_clean.append(word)

for phrase in lista_train_all:
    phrase_clean = cleanup(phrase)
    phrase_clean = phrase_clean.split()
    for word in phrase_clean:
        train_clean.append(word)

pos_clean = pd.Series(pos_clean)
neg_clean = pd.Series(neg_clean)
irr_clean = pd.Series(irr_clean)
train_clean = pd.Series(train_clean)

In [354]:
prob_pos = len(pos_clean)/len(train_clean)
prob_neg = len(neg_clean)/len(train_clean)
prob_irr = len(irr_clean)/len(train_clean)

In [355]:
times_pos = pos_clean.value_counts()
times_neg = neg_clean.value_counts()
times_irr = irr_clean.value_counts()
times_all = train_clean.value_counts()

In [356]:
def Classificador_LaPlace(phrase):
    phrase = cleanup(phrase)
    phrase = phrase.split()
    positive = 0
    negative = 0
    irrelevant = 0
    for word in phrase:
        laplace_pos = 1
        laplace_neg = 1
        laplace_irr = 1
        p = 0
        n = 0
        i = 0
        if word in times_pos:
            p = times_pos[word]
            if p != 0:
                laplace_pos *= ((p+1)/(len(pos_clean)+len(times_all)))
        if word in times_neg:
            n = times_neg[word]
            if n != 0:
                laplace_neg *= ((n+1)/(len(neg_clean)+len(times_all)))
        if word in times_irr:
            i = times_irr[word]
            if i != 0:
                laplace_irr *= ((i+1)/(len(irr_clean)+len(times_all)))
        chance_list = [laplace_pos,laplace_neg,laplace_irr]
        if laplace_pos == max(chance_list):
            positive += 1
        if laplace_neg == max(chance_list):
            negative += 1
        if laplace_irr == max(chance_list):
            irrelevant += 1
    list_chances = [positive,negative,irrelevant]
    if positive == max(list_chances):
        return 1
    if negative == max(list_chances):
        return 2
    if irrelevant == max(list_chances):
        return 0

In [357]:
bot_train = []
for phrase in train['Mensagem']:
    mensagem = Classificador_LaPlace(phrase)
    bot_train.append(mensagem)
pd.crosstab(train['Target'], bot_train, normalize=True)*100

col_0,0,1,2
Target,,,
0,0.2,0.8,20.0
1,0.2,0.0,13.6
2,14.0,4.8,46.4


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [358]:
test_pos = test.loc[test['Target'] == 1, ['Mensagem']]
test_neg = test.loc[test['Target'] == 2, ['Mensagem']]
test_irr = test.loc[test['Target'] == 0, ['Mensagem']]
test_all = test.loc[:,['Mensagem']]

In [359]:
lista_test_pos = test_pos['Mensagem'].values.tolist()
lista_test_neg = test_neg['Mensagem'].values.tolist()
lista_test_irr = test_irr['Mensagem'].values.tolist()
lista_test_all = test_all['Mensagem'].values.tolist()

pos_clean = []
neg_clean = []
irr_clean = []
test_clean = []

for phrase in lista_test_pos:
    phrase_clean = cleanup(phrase)
    phrase_clean = phrase_clean.split()
    for word in phrase_clean:
        pos_clean.append(word)

for phrase in lista_test_neg:
    phrase_clean = cleanup(phrase)
    phrase_clean = phrase_clean.split()
    for word in phrase_clean:
        neg_clean.append(word)

for phrase in lista_test_irr:
    phrase_clean = cleanup(phrase)
    phrase_clean = phrase_clean.split()
    for word in phrase_clean:
        irr_clean.append(word)

for phrase in lista_test_all:
    phrase_clean = cleanup(phrase)
    phrase_clean = phrase_clean.split()
    for word in phrase_clean:
        test_clean.append(word)

pos_clean = pd.Series(pos_clean)
neg_clean = pd.Series(neg_clean)
irr_clean = pd.Series(irr_clean)
test_clean = pd.Series(test_clean)

In [360]:
prob_pos = len(pos_clean)/len(test_clean)
prob_neg = len(neg_clean)/len(test_clean)
prob_irr = len(irr_clean)/len(test_clean)

print(prob_pos)
print(prob_neg)
print(prob_irr)

times_pos = pos_clean.value_counts()
times_neg = neg_clean.value_counts()
times_irr = irr_clean.value_counts()
times_all = test_clean.value_counts()

0.04536607276538404
0.7504117382841743
0.2042221889504417


In [361]:
def Classificador_LaPlace(phrase):
    phrase = cleanup(phrase)
    phrase = phrase.split()
    positive = 0
    negative = 0
    irrelevant = 0
    for word in phrase:
        laplace_pos = 1
        laplace_neg = 1
        laplace_irr = 1
        p = 0
        n = 0
        i = 0
        if word in times_pos:
            p = times_pos[word]
            if p != 0:
                laplace_pos *= ((p+1)/(len(pos_clean)+len(times_all)))
        if word in times_neg:
            n = times_neg[word]
            if n != 0:
                laplace_neg *= ((n+1)/(len(neg_clean)+len(times_all)))
        if word in times_irr:
            i = times_irr[word]
            if i != 0:
                laplace_irr *= ((i+1)/(len(irr_clean)+len(times_all)))
        chance_list = [laplace_pos,laplace_neg,laplace_irr]
        if laplace_pos == max(chance_list):
            positive += 1
        if laplace_neg == max(chance_list):
            negative += 1
        if laplace_irr == max(chance_list):
            irrelevant += 1
    list_chances = [positive,negative,irrelevant]
    if positive == max(list_chances):
        return 1
    if negative == max(list_chances):
        return 2
    if irrelevant == max(list_chances):
        return 0

In [362]:
bot_test = {"Target Bot": []}
for phrase in test['Mensagem']:
    mensagem = Classificador_LaPlace(phrase)
    bot_test["Target Bot"].append(mensagem)
pd.crosstab(test['Target'], bot_test['Target Bot'], normalize=True)*100

{'Target Bot': [2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 0, 1, 2, 1, 2, 1, 0, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 0, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2]}


col_0,0,1,2
Target,,,
0,0.0,4.4,14.4
1,0.0,0.0,3.6
2,1.6,58.4,17.6


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**